In [ ]:
"""Daten laden"""
# initpath=r"C:\Users\maxim\Nextcloud\Shared\Austausch_Max\Projekt_Entropie\OCV_daten"

initpath=r"C:\Users\Dominik\tubCloud\Studis\Austausch_Max\Projekt_Entropie\OCV_daten"
datei="dS24NCA10_ButlerVolmerGITT.txt"
data_raw=load(initpath,datei,skip=0,separation=" ")
data_temp=np.delete(data_raw,[1,2,3,5,10,12,13],1)
data=data_temp

In [ ]:
test=filtplot(data,0,109,plot=1,col=4)

In [ ]:
lines_iOCV_dis=[[19,20,21],[35,36,37],[51,52,53],[67,68,69],[83,84,85],[99,100,101]]
lines_iOCV_ch=[[27,28,29],[43,44,45],[59,60,61],[75,76,77],[91,92,93],[107,108,109]]

OCV={"ch":{},"dis":{}}
c_rate=["C/5","C/2","3C/4","1C","1.5C","5.1A"]



l=0
   
for i in lines_iOCV_ch:
    tempch=filtplot(data,i[0],i[2],plot=0,col=2)

    OCV["ch"][c_rate[l]]=np.asarray(tempch,dtype=np.float64)

    l+=1
    
    
l=0

for i in lines_iOCV_dis:
    tempdis=filtplot(data,i[0],i[2],plot=0,col=2)

    OCV["dis"][c_rate[l]]=np.asarray(tempdis,dtype=np.float64)

    l+=1

fit_all=[]
plot_all=[]
hppc_data=[]
IR_all=[]
for a in range(len(lines_iOCV_ch)):
    #Ladepulse(27,28)
    #Entladepulse(20,21)
    lines=[lines_iOCV_dis[a],lines_iOCV_ch[a]]
    data_t=data.copy()
    data_t[:,0]=data_t[:,0]*3600
    strome=[]
    for i in range(len(lines)):
        temp=filtplot(data_t,lines[i][1],lines[i][2],plot=0,col=3)
        temp=temp.astype(float)
        temp=consecutive_neu(temp) 
        # for i in range(len(temp)):
        #     temp[i]=process_array(temp[i])
        hppc_data.append(temp)
    plot_data,fit_data,IR_data=[],[],[]
    for i in range(len(lines)):
        plot_temp,fit_temp,strom,IR =hppc_fit_grenzen(data_t,lines[i],0)
        fit_temp=np.vstack([fit_temp])
        plot_data.append(plot_temp)
        fit_data.append(fit_temp)
        IR_data.append(IR)
    plot_all.append(plot_data)
    fit_all.append(fit_data)    
    IR_all.append(IR_data)


In [ ]:
fig,axes=plt.subplots(2,2)
axes[0,0].plot(fit_all[0][0][1,:])
axes[1,0].plot(fit_all[0][0][2,:])
axes[0,1].plot(fit_all[0][0][3,:])
axes[1,1].plot(fit_all[0][0][4,:])
axes[0,0].plot(np.flip(fit_all[0][1][1,:]))
axes[1,0].plot(np.flip(fit_all[0][1][2,:]))
axes[0,1].plot(np.flip(fit_all[0][1][3,:]))
axes[1,1].plot(np.flip(fit_all[0][1][4,:]))
plt.show()

In [ ]:
d=10
plt.plot(np.linspace(hppc_data[0][d][0,0],hppc_data[0][d][-1,0],len(plot_all[0][0][d])),plot_all[0][0][d][:])
plt.plot(hppc_data[0][d][:,0],hppc_data[0][d][:,2]-hppc_data[0][d][0,2])
plt.show()

In [ ]:
"""Butler-Volmer"""
moddata={}
for direction in OCV.keys():
    t=0
    for c in OCV[direction].keys():
        moddata[f"{c}"+" "+f"{direction}"]={}
        if direction=="ch":
            moddata[f"{c}"+" "+f"{direction}"]["U"]=IR_all[t][1]
        elif direction=="dis":
            moddata[f"{c}"+" "+f"{direction}"]["U"]=np.array([IR_all[t][0][0,:],-IR_all[t][0][1,:]])
        else:
            print("falsche Laderichtung")
        moddata[f"{c}"+" "+f"{direction}"]["I"]=OCV[direction][c][5,3]
        # moddata[f"{c}"+" "+f"{direction}"]["I"]=np.mean(OCV[direction][c][1:10,3])

        t+=1

In [ ]:
import copy

testdict = copy.deepcopy(moddata)
testdict2=copy.deepcopy(testdict)

minvol=[]
maxvol=[]    
for i in testdict.keys():
    minvol.append(min(testdict[i]["U"][0,:]))
    maxvol.append(max(testdict[i]["U"][0,:]))
commonvol=np.linspace(max(minvol),min(maxvol),100)
for i in testdict.keys():
    
    f1=interp1d(testdict[i]["U"][0,:],testdict[i]["U"][1,:],kind='linear')
    interpU=f1(commonvol)
    testdict2[i]["U"]=np.array([commonvol,interpU])

In [ ]:
# Butler Volmer testen
# e=10
# U_data=np.array([testdict2["1C dis"]["U"][1,e],
#                  testdict2["3C/4 dis"]["U"][1,e],testdict2["C/2 dis"]["U"][1,e],testdict2["C/5 dis"]["U"][1,e],
#                  0,
#                  testdict2["C/5 ch"]["U"][1,e],testdict2["C/2 ch"]["U"][1,e],testdict2["3C/4 ch"]["U"][1,e],
#                  testdict2["1C ch"]["U"][1,e]])
# #U_data=[-0.478901,-0.018507,0,0.0198429,0.048844]
# I_data=np.array([testdict2["1C dis"]["I"],
#                  testdict2["3C/4 dis"]["I"],testdict2["C/2 dis"]["I"],testdict2["C/5 dis"]["I"],
#                  0,
#                  testdict2["C/5 ch"]["I"],testdict2["C/2 ch"]["I"],testdict2["3C/4 ch"]["I"],
#                 testdict2["1C ch"]["I"]])


# p0 = [1.5, 0.5, 0.01]  # Startwerte: I0, alpha, IR
# # bounds = (
# #     [0, 0.1, 0.01],  # Untergrenzen: I0, alpha, IR
# #     [1.0, 2, 10],     # Obergrenzen: I0, alpha, IR
# # )

# # popt, pcov = curve_fit(
# #     model2, I_data, U_data,
# #     p0=p0, bounds=bounds
# # )
# solution = fmin(cost_func2,p0,args=(U_data,I_data))  
# # Ausgabe der Fit-Ergebnisse
# I0_fit, alpha_fit, R_fit = solution
# print(f"Fit-Ergebnisse:\nI0: {I0_fit:.4e}\nalpha: {alpha_fit:.4f}\nR: {R_fit:.4f}")

# # Fit-Kurve berechnen
# U_fit = np.linspace(min(U_data), max(U_data), 100)
# I_fit=np.linspace(min(I_data),max(I_data),100)
# I_fit = model2(U_fit,I_fit, solution)

# # Plot der Daten und des Fits
# plt.scatter( U_data,I_data, label="Daten", color="blue")
# plt.plot( U_fit,I_fit ,label="Fit", color="red")

# plt.legend()
# plt.grid()
# plt.title("Curve-Fit mit modifizierter Butler-Volmer-Gleichung")
# plt.show()

In [ ]:
"""Butler Volmer Fit für alle Daten"""
butler_fit=np.array([])
butler_plot={}
for a in range(len(testdict2["C/2 ch"]["U"][0,:])):
    Udata=[0]
    Idata=[0]
    for i in testdict2.keys():
        if "5.1A" not in i and "1.5C" not in i:
            Udata.append(testdict2[i]["U"][1,a])
            Idata.append(testdict2[i]["I"])
            print(i)
    Udata=np.sort(np.array(Udata))
    Idata=np.sort(np.array(Idata))
    p0 = [1.5, 0.5, 0.01]  # Startwerte: I0, alpha, IR
    # bounds = (
    #     [0, 0.1, 0.01],  # Untergrenzen: I0, alpha, IR
    #     [1.0, 2, 10],     # Obergrenzen: I0, alpha, IR
    # )

    # popt, pcov = curve_fit(
    #     model2, I_data, U_data,
    #     p0=p0, bounds=bounds
    # )
    solution = fmin(cost_func2,p0,args=(Udata,Idata))  
    # Ausgabe der Fit-Ergebnisse
    I0_fit, alpha_fit, R_fit = solution
    print(f"Fit-Ergebnisse:\nI0: {I0_fit:.4e}\nalpha: {alpha_fit:.4f}\nR: {R_fit:.4f}")

    # Fit-Kurve berechnen
    Ufit = np.linspace(min(Udata), max(Udata), 100)
    Ifit=np.linspace(min(Idata),max(Idata),100)
    Ifitted = model2(Ufit,Ifit, solution)
    
    fit_t=np.array([testdict2["C/2 ch"]["U"][0,a],I0_fit, alpha_fit, R_fit])
    plot_t=np.array([Ufit,Ifitted]).T
    if a == 0:
        butler_fit = fit_t
    else:
        butler_fit = np.column_stack([butler_fit, fit_t])
    vol=testdict2["C/2 ch"]["U"][0,a]
    butler_plot[f"{vol:.4f}"+" V"]=plot_t    

In [ ]:
e=10
U=np.array([testdict2["1C dis"]["U"][1,e],
                 testdict2["3C/4 dis"]["U"][1,e],testdict2["C/2 dis"]["U"][1,e],testdict2["C/5 dis"]["U"][1,e],
                 0,
                 testdict2["C/5 ch"]["U"][1,e],testdict2["C/2 ch"]["U"][1,e],testdict2["3C/4 ch"]["U"][1,e],
                 testdict2["1C ch"]["U"][1,e]])
#U_data=[-0.478901,-0.018507,0,0.0198429,0.048844]
I=np.array([testdict2["1C dis"]["I"],
                 testdict2["3C/4 dis"]["I"],testdict2["C/2 dis"]["I"],testdict2["C/5 dis"]["I"],
                 0,
                 testdict2["C/5 ch"]["I"],testdict2["C/2 ch"]["I"],testdict2["3C/4 ch"]["I"],
                testdict2["1C ch"]["I"]])

plt.scatter(U,I, label="Daten", color="blue")
plt.plot(butler_plot["3.0815 V"][:,0],butler_plot["3.0815 V"][:,1] ,label="Fit", color="red")

plt.legend()
plt.grid()
plt.title("Curve-Fit mit modifizierter Butler-Volmer-Gleichung")
plt.show()

plt.plot(butler_fit[0,:],butler_fit[2,:])
plt.grid()
plt.show()

In [ ]:
# U_data=np.array([moddata["5.1A dis"]["U"][1,82],moddata["1.5C dis"]["U"][1,82],moddata["1C dis"]["U"][1,82],
#                   moddata["3C/4 dis"]["U"][1,82],moddata["C/2 dis"]["U"][1,82],moddata["C/5 dis"]["U"][1,82],
#                   0,
#                   moddata["C/5 ch"]["U"][1,5],moddata["C/2 ch"]["U"][1,5],moddata["3C/4 ch"]["U"][1,5],
#                   moddata["1C ch"]["U"][1,5],moddata["1.5C ch"]["U"][1,5],moddata["5.1A ch"]["U"][1,5]])
# #U_data=[-0.478901,-0.018507,0,0.0198429,0.048844]
# I_data=np.array([moddata["5.1A dis"]["I"],moddata["1.5C dis"]["I"],moddata["1C dis"]["I"],
#                   moddata["3C/4 dis"]["I"],moddata["C/2 dis"]["I"],moddata["C/5 dis"]["I"],
#                   0,
#                   moddata["C/5 ch"]["I"],moddata["C/2 ch"]["I"],moddata["3C/4 ch"]["I"],
#                   moddata["1C ch"]["I"],moddata["1.5C ch"]["I"],moddata["5.1A ch"]["I"]])


# p0 = [0.4, 0.3, 0.01]  # Startwerte: I0, alpha, IR
# # bounds = (
# #     [0, 0.1, 0.01],  # Untergrenzen: I0, alpha, IR
# #     [1.0, 2, 10],     # Obergrenzen: I0, alpha, IR
# # )

# # popt, pcov = curve_fit(
# #     model2, I_data, U_data,
# #     p0=p0, bounds=bounds
# # )
# solution = fmin(cost_func2,p0,args=(U_data,I_data))  
# # Ausgabe der Fit-Ergebnisse
# I0_fit, alpha_fit, R_fit = solution
# print(f"Fit-Ergebnisse:\nI0: {I0_fit:.4e}\nalpha: {alpha_fit:.4f}\nR: {R_fit:.4f}")

# # Fit-Kurve berechnen
# U_fit = np.linspace(min(U_data), max(U_data), 100)
# I_fit=np.linspace(min(I_data),max(I_data),100)
# I_fit = model2(U_fit,I_fit, solution)

# # Plot der Daten und des Fits
# plt.scatter( U_data,I_data, label="Daten", color="blue")
# plt.plot( U_fit,I_fit ,label="Fit", color="red")

# plt.legend()
# plt.grid()
# plt.title("Curve-Fit mit modifizierter Butler-Volmer-Gleichung")
# plt.show()